# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
print(stocks)

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [9]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 66374592, 'calculationPrice': 'tops', 'change': 0.68, 'changePercent': 0.00394, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 173.06, 'iexAskSize': 200, 'iexBidPrice': 173.04, 'iexBidSize': 300, 'iexClose': 173.08, 'iexCloseTime': 1696435051013, 'iexLastUpdated': 1696435051013, 'iexMarketPercent': 0.022613401359806693, 'iexOpen': 171.14, 'iexOpenTime': 1696426200014, 'iexRealtimePrice': 173.08, 'iexRealtimeSize': 74, 'iexVolume': 533965, 'lastTradeTime': 1696435051013, 'latestPrice': 173.08, 'latestSource': 'IEX real time price', 'latestTime': '11:57:31 AM', 'latestUpdate': 1696435051013, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2705972874560, 'odd

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [12]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap/ 1000000000000)

2.70597287456


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [17]:
cols = ['ticker', 'stock_price', 'market_cap', 'no_of_shares_to_buy']
final_df = pd.DataFrame(columns = cols)

final_df

,ticker,stock_price,market_cap,no_of_shares_to_buy


In [23]:
final_df.append(
    pd.Series(
    [
    symbol,
    price,
    market_cap,
    'N/A'
    ],
    index = cols
    ),
    ignore_index = True
)

,ticker,stock_price,market_cap,no_of_shares_to_buy
0,AAPL,173.08,2705972874560,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [46]:
final_df = pd.DataFrame(columns = cols)

for stock in stocks['Ticker']:
    try:
        api_url = f'https://api.iex.cloud/v1/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
        final_df = final_df.append(
            pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index = cols),
            ignore_index = True
        )
    except ValueError as e:
        continue

In [47]:
final_df

,ticker,stock_price,market_cap,no_of_shares_to_buy
0,A,110.890,32445002703,N/A
1,AAL,12.665,8274835252,N/A
2,AAP,52.230,3106131732,N/A
3,AAPL,172.620,2698781127840,N/A
4,ABBV,146.705,258941173189,N/A
...,...,...,...,...
489,YUM,122.985,34461784394,N/A
490,ZBH,109.360,22852317694,N/A
491,ZBRA,229.385,11776250626,N/A
492,ZION,32.930,4878419230,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [41]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i: i + n]

In [64]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
final_df = pd.DataFrame(columns = cols)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://api.iex.cloud/v1/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        try:
            final_df = final_df.append(
                pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = cols),
                ignore_index = True
            )
        except KeyError as e:
            continue
final_df

,ticker,stock_price,market_cap,no_of_shares_to_buy
0,A,110.795,32417206912,N/A
1,AAL,12.640,8258501191,N/A
2,AAP,52.075,3096913842,N/A
3,AAPL,172.540,2697530389280,N/A
4,ABBV,146.620,258791144222,N/A
...,...,...,...,...
491,YUM,122.880,34432362209,N/A
492,ZBH,109.170,22812614509,N/A
493,ZBRA,227.955,11702836766,N/A
494,ZION,32.990,4887307938,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [90]:
portfolio_size = input('Enter value of your portfolio: ')

Enter value of your portfolio: 1000000


In [91]:
try:
    val = float(portfolio_size)
except ValueError:
    portfolio_size = input('Reenter value of your portfolio: ')
#not proper error handling

In [96]:
position_size = val/len(final_df.index)
# number_ofapple_shares = position_size/500
# print(math.floor(number_ofapple_shares))

for i in range(len(final_df.index)):
    final_df.loc[i, 'no_of_shares_to_buy'] = math.floor(position_size/final_df.loc[i, 'stock_price'])
    
print(final_df)

    ticker  stock_price     market_cap no_of_shares_to_buy
0        A      110.795    32417206912                  18
1      AAL       12.640     8258501191                 159
2      AAP       52.075     3096913842                  38
3     AAPL      172.540  2697530389280                  11
4     ABBV      146.620   258791144222                  13
..     ...          ...            ...                 ...
491    YUM      122.880    34432362209                  16
492    ZBH      109.170    22812614509                  18
493   ZBRA      227.955    11702836766                   8
494   ZION       32.990     4887307938                  61
495    ZTS      170.030    78267683357                  11

[496 rows x 4 columns]


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [112]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_df.to_excel(writer, 'Recommended_Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [113]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': 0,
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [103]:
writer.sheets['Recommended_Trades'].set_column('A:A', 18, string_format)
writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [115]:
column_formats = {
    'A': ['ticker', string_format],
    'B': ['stock_price', dollar_format],
    'C': ['market_cap', dollar_format], 
    'D': ['no_of_shares_to_buy', integer_format]
}

for col in column_formats:
    writer.sheets['Recommended_Trades'].set_column(f'{col}:{col}', 18, column_formats[col][1])
writer.save()

## Saving Our Excel Output

Saving our Excel file is very easy:

KeyError: 'Recommended Trades'